In [15]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [16]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## PCA

In [17]:
!wget https://s3.eu-central-1.amazonaws.com/dsr-data/UScrime/UScrime2-colsLotsOfNAremoved.csv

crimes = sqlc.read.format("com.databricks.spark.csv") \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("UScrime2-colsLotsOfNAremoved.csv")

--2017-10-15 11:05:30--  https://s3.eu-central-1.amazonaws.com/dsr-data/UScrime/UScrime2-colsLotsOfNAremoved.csv
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.72.44
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.72.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971758 (949K) [application/octet-stream]
Saving to: ‘UScrime2-colsLotsOfNAremoved.csv.1’

UScrime2-colsLotsOf 100%[===================>] 948,98K   595KB/s    in 1,6s    

2017-10-15 11:05:32 (595 KB/s) - ‘UScrime2-colsLotsOfNAremoved.csv.1’ saved [971758/971758]



In [19]:
crimes.toPandas()

,community,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,ViolentCrimesPerPop
0,Lakewoodcity,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,...,0.00,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.20
1,Tukwilacity,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,...,0.00,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.67
2,Aberdeentown,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,...,0.00,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.43
3,Willingborotownship,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,...,0.00,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.12
4,Bethlehemtownship,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,...,0.00,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.03
5,SouthPasadenacity,0.02,0.28,0.06,0.54,1.00,0.25,0.31,0.48,0.27,...,0.00,0.70,0.42,0.49,0.73,0.64,0.01,0.58,0.10,0.14
6,Lincolntown,0.01,0.39,0.00,0.98,0.06,0.02,0.30,0.37,0.23,...,0.00,0.15,0.81,0.77,0.91,0.84,0.05,0.08,0.06,0.03
7,Selmacity,0.01,0.74,0.03,0.46,0.20,1.00,0.52,0.55,0.36,...,0.00,0.59,0.58,0.52,0.79,0.78,0.01,0.33,0.00,0.55
8,Hendersoncity,0.03,0.34,0.20,0.84,0.02,0.00,0.38,0.45,0.28,...,0.00,0.01,0.78,0.48,0.79,0.75,0.04,0.17,0.04,0.53
9,Claytoncity,0.01,0.40,0.06,0.87,0.30,0.03,0.90,0.82,0.80,...,0.00,0.22,0.42,0.34,0.23,0.09,0.00,0.47,0.11,0.15


In [20]:
crimes = crimes.drop("OtherPerCap").drop("community")

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, Vector

In [22]:
assembler = VectorAssembler().setInputCols(crimes.columns).setOutputCol("features")

In [23]:
featuresDF = assembler.transform(crimes).select("features")

In [10]:
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol="features", outputCol="pca")

model = pca.fit(featuresDF)

pc = model.transform(featuresDF)

pc.toPandas()[:3]

,features,pca
0,"[0.19, 0.33, 0.02, 0.9, 0.12, 0.17, 0.34, 0.47...","[1.2138889197, 0.564567759337, -0.022284837106..."
1,"[0.0, 0.16, 0.12, 0.74, 0.45, 0.07, 0.26, 0.59...","[0.627985190195, 1.16689414866, -0.51416430664..."
2,"[0.0, 0.42, 0.49, 0.56, 0.17, 0.04, 0.39, 0.47...","[0.234349043189, 0.348070144228, 0.54876884160..."


In [11]:
sc.stop()